In [2]:
import sys
import yaml
from sklearn.model_selection import train_test_split

sys.path.append('../../../../')

In [3]:
from scripts.utils import load_data, preprocess_data_label_encoder, create_sequences, train_and_test, simulate_predictions, simulate_predictions_from_pretrained

No GPU available. Running on CPU.


In [4]:
from src.models.bidir_lstm_multihead import build_model

In [5]:
YAML_FILE = '../../../../configs/games/tic-tac-toe/bidir_lstm/10k.yml'

In [6]:
with open(YAML_FILE, 'r') as f:
    config = yaml.safe_load(f)
print(config)

{'name': '/games/tic-tac-toe/bidir_lstm/10k', 'model': {'type': 'Bidirectional Multihead LSTM', 'lstm_units': 128, 'num_lstm_layers': 2, 'attention': True}, 'training': {'batch_size': 64, 'learning_rate': 0.001, 'num_epochs': 5}, 'data': {'game': 'tic-tac-toe', 'sequence_length': 20, 'max_event_length': 50, 'path': '/games/tic-tac-toe/10k_single_agent.csv'}}


In [7]:
model_params = {
    'lstm_units': config['model']['lstm_units'],
    'num_lstm_layers': config['model']['num_lstm_layers'],
    'attention': config['model']['attention']
}

In [8]:
# Load and preprocess data
df = load_data(f'../../../../data/processed/{config['data']['path']}')
df, e_event, e_agent, e_context = preprocess_data_label_encoder(df)

# Create sequences
sequence_length = config['data']['sequence_length']
X, y = create_sequences(df, sequence_length)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
model = train_and_test(
    X_train, 
    X_test, 
    y_train, 
    y_test, 
    e_event, 
    e_agent, 
    e_context, 
    name=config['name'], 
    epochs=config['training']['num_epochs'], 
    batch_size=config['training']['batch_size'],
    model_params=model_params,
    build_model=build_model,
)

Epoch 1/5
1204/1204 ━━━━━━━━━━━━━━━━━━━━ 54s 40ms/step - loss: 3.1492 - output_1_accuracy: 0.8718 - output_2_accuracy: 0.6790 - output_3_accuracy: 0.1954 - val_loss: 2.1965 - val_output_1_accuracy: 0.9312 - val_output_2_accuracy: 0.9312 - val_output_3_accuracy: 0.2740
Epoch 2/5
1204/1204 ━━━━━━━━━━━━━━━━━━━━ 48s 39ms/step - loss: 2.2145 - output_1_accuracy: 0.9294 - output_2_accuracy: 0.9297 - output_3_accuracy: 0.2720 - val_loss: 2.1956 - val_output_1_accuracy: 0.9131 - val_output_2_accuracy: 0.9230 - val_output_3_accuracy: 0.2678
Epoch 3/5
1204/1204 ━━━━━━━━━━━━━━━━━━━━ 50s 41ms/step - loss: 2.1825 - output_1_accuracy: 0.9309 - output_2_accuracy: 0.9316 - output_3_accuracy: 0.2743 - val_loss: 2.1371 - val_output_1_accuracy: 0.9312 - val_output_2_accuracy: 0.9310 - val_output_3_accuracy: 0.2851
Epoch 4/5
1204/1204 ━━━━━━━━━━━━━━━━━━━━ 47s 39ms/step - loss: 2.1269 - output_1_accuracy: 0.9326 - output_2_accuracy: 0.9328 - output_3_accuracy: 0.2842 - val_loss: 2.1526 - val_output_1_accur

Test Results:
loss: 2.0587
compile_metrics: 0.9315


In [10]:
simulate_predictions(
    data=X,
    model=model,
    e_event=e_event,
    e_agent=e_agent,
    e_context=e_context,
    n = 15,
    k = config['data']['sequence_length']
)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 524ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Initial sequence:
MOVE, O, 2,2
MOVE, X, 0,2
MOVE, O, 1,1
MOVE, X, 1,0
MOVE, O, 0,0
GAME_END, system, O
GAME_START, system, New Game
MOVE, X, 1,2
MOVE, O, 1,1
MOVE, X, 0,2
MOVE, O, 0,0
MOVE, X, 0,1
MOVE, O, 1,0
MOVE, X, 2,1
MOVE, O, 2,0
GAME_END, system, O
GAME_START, system, New Game
MOVE, X, 1,1
MOVE, O, 0,1
MOVE, X, 2,2

Predicted moves:
Move 1: MOVE, O, 0,0
Move 2: MOVE, X, 1,0
Move 3: MOVE, O, 2,0
Move 4: MOVE, X, 0,2
Mov

In [11]:
simulate_predictions_from_pretrained(
    data=X,
    modelpath=f'../../../../models/{config['name']}.h5',
    e_event=e_event,
    e_agent=e_agent,
    e_context=e_context,
    n = 15,
    k = config['data']['sequence_length']
)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 647ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Initial sequence:
MOVE, X, 0,0
MOVE, O, 2,1
MOVE, X, 0,1
GAME_END, system, X
GAME_START, system, New Game
MOVE, X, 1,0
MOVE, O, 1,1
MOVE, X, 2,1
MOVE, O, 1,2
MOVE, X, 0,0
MOVE, O, 0,1
MOVE, X, 2,0
GAME_END, system, X
GAME_START, system, New Game
MOVE, X, 0,1
MOVE, O, 1,2
MOVE, X, 0,2
MOVE, O, 0,0
MOVE, X, 1,1
MOVE, O, 2,0

Predicted moves:
Move 1: MOVE, X, 2,0
Move 2: MOVE, O, X
Move 3: MOVE, X, 1,1
Move 4: GAME_END, system, 